# Gathering Data

In [1]:
#EDIT ME to change target subreddit...
# target_subreddit_list = ['Naruto', 'BokuNoHeroAcademia', 'anime', 'manga', 'comicbooks', 'StarWars', 'startrek']
# target_subreddit_list = ['LonghornNation', 'cfb', 'Genshin_Impact']
target_subreddit_list = ['comicbooks']
target_num_posts = 5000
scrape_type = "submission"  # set to "submission" or "comment"

In [2]:
#Imports
import requests

import pandas as pd

In [3]:
url = f"https://api.pushshift.io/reddit/search/{scrape_type}"

In [4]:
def my_get_data(subreddit, num_posts):
    joined_df = pd.DataFrame()
    
    my_changing_utc = None #1677703953
    
    max_num_posts = 500
    remaining_num = num_posts
       
    i = 0
    while i < num_posts:
        
        if remaining_num < max_num_posts:
            temp_size = remaining_num
        else:
            temp_size = max_num_posts
            remaining_num -= temp_size
        
        params = {
            'subreddit': subreddit,
            'size':temp_size,
            'before': my_changing_utc
        }

        breakflag = False
        res = requests.get(url, params)
        if res.status_code < 400:
            data = res.json()
            posts = data['data']
            df = pd.DataFrame(posts)
        else:
            print(f"CAUTION: During requests.get, received code: {res.status_code}")
            breakflag = True
            break
        
        if breakflag == False:
            if i == 0:
                joined_df = df
            else:
                joined_df = pd.concat([joined_df, df], axis=0)
            i += temp_size

        my_changing_utc = df['created_utc'].min()  # .min() suggestion from Kate Skibo

        
    
    temp = joined_df[joined_df['subreddit'].str.lower() == subreddit.lower()] #drop entries with errant subreddit
    
    if scrape_type == "submission":
        joined_df = temp.drop_duplicates(subset='title')            #drop_duplicates idea from Jack Vaughan
    elif scrape_type == "comment":
        joined_df = temp.drop_duplicates(subset=['author','body'])  #drop_duplicates criteria from Jack Vaughan
    
    return joined_df

In [5]:
for target_subreddit in target_subreddit_list:
    joined_df = my_get_data(target_subreddit, target_num_posts)
    joined_df.to_csv(f'./data/{target_subreddit}_{scrape_type}s.csv')
    print(f"{joined_df.shape[0]} posts scraped from r/{target_subreddit}")

4845 posts scraped from r/comicbooks
